In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import statsmodels.api as sm
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

# Load the dataset

In [34]:
data = pd.read_csv("C:\\Users\\Bala Vignesh.A\\Desktop\\SCMA 632\\NSSO68.csv", low_memory=False)

In [35]:
display (data)

,slno,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,...,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot,non_veg
0,1,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2,12.000000,154.180000,1
1,2,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2,333.000000,484.950000,1
2,3,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2,35.000000,214.840000,1
3,4,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2,168.333333,302.300000,1
4,5,4.10E+31,1,41000,68,10,1,2,24,242,...,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2,15.000000,148.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101657,101658,8.00E+31,1,79998,68,10,1,1,1,12,...,0.0,0.0,0.000000,544.013667,28.441750,J$K,2,0.000000,25.833333,1
101658,101659,8.00E+31,1,79998,68,10,1,1,1,12,...,0.0,0.0,8.000000,417.616600,25.490282,J$K,2,0.000000,49.000000,1
101659,101660,8.00E+31,1,79998,68,10,1,1,1,12,...,0.0,0.0,7.142857,378.300429,25.800107,J$K,2,0.000000,32.285714,1
101660,101661,8.00E+31,1,79998,68,10,1,1,1,12,...,0.0,0.0,14.000000,510.023600,30.220170,J$K,2,0.000000,39.200000,1


In [36]:
print(data.columns)

Index(['slno', 'grp', 'Round_Centre', 'FSU_number', 'Round', 'Schedule_Number',
       'Sample', 'Sector', 'state', 'State_Region',
       ...
       'sauce_jam_v', 'Othrprocessed_v', 'Beveragestotal_v', 'foodtotal_v',
       'foodtotal_q', 'state_1', 'Region', 'fruits_df_tt_v', 'fv_tot',
       'non_veg'],
      dtype='object', length=385)


# Define the dependent variable (non_veg) and independent variables

In [37]:
y = data['foodtotal_v']  
X = data[['sauce_jam_v', 'Othrprocessed_v', 'Beveragestotal_v', 'fv_tot']]  

# Custom Tobit model implementation

In [38]:
class TobitModel:
    def __init__(self, endog, exog, lower=None, upper=None):
        self.endog = endog
        self.exog = exog
        self.lower = lower
        self.upper = upper

    def loglik(self, params):
        beta = params[:-1]
        sigma = params[-1]
        mu = np.dot(self.exog, beta)
        
        # Ensure sigma is positive
        sigma = np.abs(sigma) + 1e-10
        
        # Calculate the log-likelihood
        llf = np.zeros_like(self.endog, dtype=float)

        # Censored from below
        if self.lower is not None:
            llf = np.where(
                self.endog == self.lower,
                np.log(np.clip(norm.cdf((self.lower - mu) / sigma), 1e-10, 1)),
                llf
            )

        # Censored from above
        if self.upper is not None:
            llf = np.where(
                self.endog == self.upper,
                np.log(np.clip(1 - norm.cdf((self.upper - mu) / sigma), 1e-10, 1)),
                llf
            )

        # Uncensored
        uncensored = (self.endog > self.lower) & (self.endog < self.upper)
        llf[uncensored] = -0.5 * np.log(2 * np.pi) - np.log(sigma) - (self.endog[uncensored] - mu[uncensored]) ** 2 / (2 * sigma ** 2)

        return -np.sum(llf)

    def fit(self):
        start_params = np.append(np.zeros(self.exog.shape[1]), 1)
        res = minimize(self.loglik, start_params, method='L-BFGS-B')
        return res

# Prepare the data for Tobit regression

In [39]:
y_tobit = np.clip(y, 0, 1)

In [40]:
X_tobit = sm.add_constant(X)

# Fit the Tobit model


In [41]:
model = TobitModel(y_tobit, X_tobit, lower=0, upper=1)
results = model.fit()

# Print the Tobit model results

In [42]:
print("Tobit Model Results:")

Tobit Model Results:


In [43]:
print(results)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1630.8623801597212
        x: [-3.332e+02  1.674e+02  1.565e+02  1.236e+04  1.675e+02
             3.541e+03]
      nit: 67
      jac: [-3.342e-03  0.000e+00  0.000e+00 -4.547e-05 -2.724e-02
             1.114e-03]
     nfev: 805
     njev: 115
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>


The output  indicates that the Tobit model has converged successfully. Here’s a breakdown of the key information:

CONVERGENCE: The algorithm has converged, meaning it has found parameter estimates that optimize the log-likelihood function.
REL_REDUCTION_OF_F_<=_FACTR*EPSMCH: This typically indicates that the relative reduction in the objective function (log-likelihood in this case) is less than the specified threshold (factr * epsmch), which is a criterion for convergence in the optimization process.
success: True, indicating that the optimization was successful.
fun: The final value of the objective function (negative log-likelihood), which in this case is approximately 1630.86.
x: The estimated parameters of the Tobit model. The array [ -3.332e+02, 1.674e+02, 1.565e+02, 1.236e+04, 1.675e+02, 3.541e+03] represents the estimated coefficients for your intercept and independent variables.
nit: Number of iterations performed during optimization.
jac: Final gradient vector (Jacobian) of the objective function at the solution.
nfev: Number of evaluations of the objective function.
njev: Number of evaluations of the Jacobian (gradient).
This output confirms that your Tobit model has been fitted successfully to your data. You can interpret the coefficients (x values) as the estimated effects of each independent variable on the dependent variable, adjusted for the Tobit model's censored nature.